In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_csv('apple_stock_data.csv')

In [4]:
df.head()

,Date,AAPL_Adj_Close,META_Adj_Close,AMZN_Adj_Close,GOOGL_Adj_Close,MSFT_Adj_Close,GDP,CPI,Unemployment_Rate,Interest_Rate,Industrial_Production,Retail_Sales,Personal_Saving_Rate,ts_polarity,twitter_volume
0,2016-01-01,23.871531,103.737661,33.308249,38.572865,48.884937,18525.933,236.916,4.8,2.09,99.4391,447376.0,6.1,0.119693,417
1,2016-01-02,23.876633,103.129490,32.822000,38.341312,48.734684,18525.933,236.916,4.8,2.09,99.4391,447376.0,6.1,0.140774,495
2,2016-01-03,23.881735,102.521318,32.335750,38.109760,48.584431,18525.933,236.916,4.8,2.09,99.4391,447376.0,6.1,0.181132,518
3,2016-01-04,23.886837,101.913147,31.849501,37.878208,48.434177,18525.933,236.916,4.8,2.09,99.4391,447376.0,6.1,0.070389,1133
4,2016-01-05,23.288244,102.421623,31.689501,37.982449,48.655136,18525.933,236.916,4.8,2.09,99.4391,447376.0,6.1,0.133635,1430


In [5]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Define features and targets
features = [
    'AMZN_Adj_Close', 'GOOGL_Adj_Close', 'MSFT_Adj_Close', 'GDP', 'CPI',
    'Unemployment_Rate', 'Interest_Rate', 'Industrial_Production',
    'Retail_Sales', 'Personal_Saving_Rate', 'ts_polarity', 'twitter_volume',
    'Day', 'Month', 'Year'
]
target1 = 'AAPL_Adj_Close'
target2 = 'META_Adj_Close'

# Preprocess the 'Date' column in the DataFrame
df['Date'] = pd.to_datetime(df['Date'])
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# Split the dataset into features (X) and targets (y)
X = df[features]
y1 = df[target1]
y2 = df[target2]

# Split the dataset into training and testing sets
X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(X, y1, y2, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the model
input_layer = Input(shape=(len(features),))

# Shared hidden layers
shared_layer = Dense(64, activation='relu')(input_layer)
shared_layer = Dense(32, activation='relu')(shared_layer)

# Output layers
output1 = Dense(1, name='AAPL_Output')(shared_layer)
output2 = Dense(1, name='META_Output')(shared_layer)

# Define the model
model = Model(inputs=input_layer, outputs=[output1, output2])

# Compile the model
model.compile(optimizer='adam',
              loss={'AAPL_Output': 'mse', 'META_Output': 'mse'},
              metrics={'AAPL_Output': 'mae', 'META_Output': 'mae'})

# Train the model
model.fit(X_train, {'AAPL_Output': y1_train, 'META_Output': y2_train},
          validation_data=(X_test, {'AAPL_Output': y1_test, 'META_Output': y2_test}),
          epochs=50, batch_size=32)

# Function to get predictions from user inputs
def predict_prices():
    print("Please enter the following inputs:")
    user_inputs = []

    # Get Date input and extract Day, Month, Year
    date_input = input("Enter Date (YYYY-MM-DD): ")
    date = pd.to_datetime(date_input)
    user_inputs.append(date.day)    # Day
    user_inputs.append(date.month)  # Month
    user_inputs.append(date.year)   # Year

    # Get other feature inputs
    for feature in features[:-3]:  # Exclude 'Day', 'Month', 'Year' from this loop
        user_input = float(input(f"{feature}: "))
        user_inputs.append(user_input)

    # Scale the inputs
    user_inputs = scaler.transform([user_inputs])

    # Predict using the model
    predictions = model.predict(user_inputs)
    print(f"Predicted AAPL_Adj_Close: {predictions[0][0][0]:.2f}")
    print(f"Predicted META_Adj_Close: {predictions[1][0][0]:.2f}")

# Call the prediction function
predict_prices()

NotFoundError: dlopen(/Users/kalp/anaconda3/lib/python3.11/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 0x0006): Symbol not found: __ZN3tsl8internal10LogMessageC1EPKcii
  Referenced from: <D2EF42E3-3A7F-39DD-9982-FB6BCDC2853C> /Users/kalp/anaconda3/lib/python3.11/site-packages/tensorflow-plugins/libmetal_plugin.dylib
  Expected in:     <2814A58E-D752-317B-8040-131217E2F9AA> /Users/kalp/anaconda3/lib/python3.11/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so